# Aula 4 - Exercícios Extra - Pandas e Fontes de dados


In [1]:
# Importe as bibliotecas pandas, numpy, glob
import pandas as pd
import numpy as np


### 4.6 Escreva uma função para gerar uma permutação (aleatória) de uma lista e colocar o resultado num dataframe com os elementos da lista agrupados em grupos de tamanho N (Ex.: N=4). 
#### A coluna de índices deve conter o nome dos grupos. Os nomes dos grupos devem seguir a seguinte regra: Grupo 1, Grupo 2, Grupo 3,..., Grupo K.
#### Veja como ficaria o resultado para o exemplo a seguir com uma lista de 9 elementos e tamanho N=4.

|         | 0                  |
|:--------|:-------------------|
| Grupo 0 | Julie Lewis        |
| Grupo 0 | Mary Phinney       |
| Grupo 0 | Gilbert Ravenell   |
| Grupo 0 | Jillian Brennan    |
| Grupo 1 | Michelle Humphreys |
| Grupo 1 | Kimberly Ransome   |
| Grupo 1 | Lisa Watson        |
| Grupo 1 | Jennifer Daniels   |
| Grupo 2 | Olga Crocker       |

In [14]:
# Dica 1: A função len calcula o tamanho de uma lista.
# Dica 2: Use a função math.ceil(N/n), onde N é o tamanho da lista e n é a quantidade
#         de pessoas em cada grupo

def create_groups(names_list, n):
    """Crie um dataframe com os nomes da lista names_list agrupados em grupos de tamanho n.
    """
    ### BEGIN SOLUTION
    from random import shuffle
    import math
    import pandas as pd
    N = len(names_list)
    result_groups = []
    groups = ['Grupo {0}'.format(g) for g in range(math.ceil(N/n))]*group_length 
    shuffle(names_list)
    groups.sort()
    return pd.DataFrame(index=groups[:N], data=names_list)
    ### END SOLUTION


In [15]:
## Faça seus testes aqui



In [16]:
# Validação
# Instale a biblioteca names, na primeira vez que rodar esta célula
#!pip install names
import names
N = 9
group_length = 4
test_data = [names.get_full_name() for n in range(N)]
result = ['Grupo 0', 'Grupo 0', 'Grupo 0', 'Grupo 0', 'Grupo 1', 'Grupo 1', 'Grupo 1', 'Grupo 1', 'Grupo 2']
df = create_groups(test_data, group_length)

assert df.index.to_list() == result

### 4.7 Escreva uma função para colocar num mesmo dataframe os dados de PIB percapita e índice de gini dos municípios brasieiros. 
#### Retorne os registros completos pelo menos os atributos 'pib_percapita', 'cod_ibge6', '1991', '2000' e '2010' dos top 10 registros com maiores PIB percapita.

#### Sugestão: Analise a diferença entre os quartis (do PIB percapita e do índice de gini) dos top 10 municípios, em termos de PIB percapita,  e os quartis de todo o brasil para estas mesmas duas variáveis. Responda: um PIB percapita significativamente maior, tem ajudado a melhorar o índice de Gini nesses municípios ? Dica: use o método describe(). 



In [17]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [23]:
# Dica 1: Use as funções criadas nos exercícios 4, 5 e 6 desta aula.
# Dica 2: Ao ler o arquivo CSV do índice de gini, se preciso, use encoding='utf8' como parâmetro do método read_csv

def top10_municipios(path_pib, path_gini, path_pop):
    """retorne um dataframe contendo os 10 municipios com maiores PIB percapita. O dataframe deve conter também 
    o valor do índice de gini e nome, UF e código ibge do respectivo município.
    """
    ### BEGIN SOLUTION
    pib_pop = merge_pop_pib(path_pop, path_pib)
    pib_percapita_2017 = pib_percapita(pib_pop)
    pib_percapita_2017.loc[:, 'cod_ibge6'] = pib_percapita_2017.loc[:,'cod_ibge7'].apply(lambda x: x[:-1])

    df_gini = pd.read_csv(path_gini, sep=';', skiprows=2, skipfooter=2, encoding='utf8', engine='python', decimal=',', na_values='...')
    df_gini.loc[:, 'cod_ibge6'] = df_gini.loc[:,'Município'].apply(lambda x: x.split(' ')[0])
    df_gini.loc[:, 'Município'] = df_gini.loc[:,'Município'].apply(lambda x: ' '.join(x.split(' ')[1:]))

    pib_gini = pib_percapita_2017.merge(df_gini, on='cod_ibge6', how='outer')
    df_top10 = pib_gini.sort_values(by='pib_percapita', ascending=False).iloc[:10,:]

    return df_top10
    ### END SOLUTION


In [22]:
# Importar e carregar as funcoes dos exercicios da aula 4
# !pip install ipynb
%cd /content/drive/MyDrive/Colab\ Notebooks
%pwd
from ipynb.fs.defs.Aula4_Exercicio_Solucoes import merge_pop_pib, pib_percapita

/content/drive/MyDrive/Colab Notebooks


In [ ]:
## Faça seus testes aqui
path_pib = 'https://github.com/alexlopespereira/curso_ciencia_dados2021/raw/master/data/originais/pib/pib_municipios.xlsx'
path_gini = 'https://github.com/alexlopespereira/curso_ciencia_dados2021/raw/master/data/originais/gini/ginibr.csv'
path_pop= 'https://github.com/alexlopespereira/curso_ciencia_dados2021/raw/master/data/originais/populacao/estimativa_dou_2017.xlsx'


In [26]:
# Validação
# Importar e carregar as funcoes dos exercicios da aula 4
from ipynb.fs.defs.Aula4_Exercicio_Solucoes import merge_pop_pib, pib_percapita

path_pib = 'https://github.com/alexlopespereira/curso_ciencia_dados2021/raw/master/data/originais/pib/pib_municipios.xlsx'
path_gini = 'https://github.com/alexlopespereira/curso_ciencia_dados2021/raw/master/data/originais/gini/ginibr.csv'
path_pop='https://github.com/alexlopespereira/curso_ciencia_dados2021/raw/master/data/originais/populacao/estimativa_dou_2017.xlsx'

test_data = {'pib_percapita': 344.8, 'cod_ibge6': '353650',
             '1991': 0.5, '2000': 0.6, '2010': 0.5}
top1 = top10_municipios(path_pib, path_gini, path_pop).iloc[0]
top1[['pib_percapita', '1991', '2000', '2010']] = \
    top1[['pib_percapita', '1991', '2000', '2010']].apply(lambda x: round(x,1))
top1dic = top1.to_dict()
dic = {k: top1dic[k] for k in ('pib_percapita', 'cod_ibge6', '1991', '2000', '2010')}
assert dic == test_data